In [ ]:
import os
from zipfile import ZipFile
import nltk
nltk.download('punkt')
nltk.download("stopwords")
import re 
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))
class Bigram_Index:
  def __init__(self):
    self.index_dict=dict()# dictionary to keep the inverted index
    self.token_list=list()# list containing tokens for each doc
    self.freq_dict=dict()# storing frequency of each term
    self.re=re
    self.nltk=nltk
    self.nltk.download("punkt")
    self.nltk.download("stopwords")
    self.stop_words=list(stopwords.words("english"))


  def make_bigram(self):
    if not(os.path.isdir('temp_dir1')):
      os.mkdir('temp_dir1')
    with ZipFile("/content/CSE508_Winter2023_Dataset2_preprocessed.zip", 'r') as zObject:    # Extracting all the members of the zip 
    # into a specific location.
      zObject.extractall(path="temp_dir1")
    dir_path='/content/temp_dir1/content/temp_dir/CSE508_Winter2023_Dataset'
    files_list=os.listdir(dir_path)
    file_tok_list=list()
    self.mapping_dict=dict()
    k=0
    for fl in files_list:
      fl_path=os.path.join(dir_path,fl)
      fobj=open(fl_path,'r')
      self.mapping_dict[k]=fl
      st=fobj.read()
      word_toks=list(nltk.bigrams(st.split(",")))
      self.token_list.append(word_toks)
      fobj.close()
      k=k+1

  def make_index(self):
    fin_token_list=set()
    #index_dict=dict()
    self.make_bigram()
    for toks in self.token_list:
      fin_token_list=fin_token_list.union(set(toks))
    fin_token_list=list(fin_token_list)
    #print(fin_token_list)
    for tok in fin_token_list:
      self.index_dict[' '.join(tok)]=list()
      self.freq_dict[' '.join(tok)]=0
    for doc in range(len(self.token_list)):
      #print(self.token_list[doc])
      for bg in self.token_list[doc]:
          #print(bg)
          if doc not in self.index_dict[' '.join(bg)]:
            self.index_dict[' '.join(bg)].append(doc)
          self.freq_dict[' '.join(bg)]=self.freq_dict[' '.join(bg)]+1
      myKeys = list(self.index_dict.keys())
      myKeys.sort()
      self.index_dict = {i: self.index_dict[i] for i in myKeys}    
      self.freq_dict={i: self.freq_dict[i] for i in myKeys}    

      #index_dict[self.token_list[doc][pos]]
    #return self.index_dict

  def preprocess_query(self,query):
    query = query.lower()
    words = self.re.sub('[^\w\s]',' ',query)
    tokens = self.nltk.word_tokenize(words)
    new_tokens = [t for t in tokens if t not in self.stop_words]
    words_list = [t for t in new_tokens if t!=' ']
    return words_list

  def process_query(self,query_text):
    q_processed=self.preprocess_query(query_text)
    q_toks=list(nltk.bigrams(q_processed))
    pos_list_set=set()
    for i in range(len(q_toks)):
      if ' '.join(q_toks[i]) in self.index_dict.keys():
          if i==0:
            pos_list_set=pos_list_set.union(set(self.index_dict[' '.join(q_toks[i])]))
          else:
            pos_list_set=pos_list_set.intersection(set(self.index_dict[' '.join(q_toks[i])]))
          doc_names_list=[self.mapping_dict[pos] for pos in pos_list_set]
          return pos_list_set,len(pos_list_set),doc_names_list
      else:
        return {},0,[]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploads=files.upload()

Saving CSE508_Winter2023_Dataset2_preprocessed.zip to CSE508_Winter2023_Dataset2_preprocessed.zip


In [ ]:
bi=Bigram_Index()
bi.make_index()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
"""import pickle as pkl
f1=open("bigram_index.pkl",'wb')
pkl.dump(bi,f1)
f1.close()"""
from dill import dump,load
f3=open("bigram_index_dill.pkl",'wb')
dump(bi,f3)
f3.close()

In [ ]:
#files.download("bigram_index.pkl")
files.download("bigram_index_dill.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle as pkl
f2=open("bigram_index.pkl",'rb')
bi_new=pkl.load(f2)
f2.close()

In [ ]:
print(bi_new.index_dict['heated wings'])

[875]


In [ ]:
print(bi_new.process_query("stressing heated wings"))

({875}, 1, ['cranfield0013'])
